# Milestone P4

---
### Context

In the current notebook, our aim is to find significant results to complement the research paper **Friendship and Mobility: User Movement in Location-Based Social Networks**. In particular, we want to compare by countries friendship behaviours.

---
### The data

owalla and Brightkite Location-based online social datasets from the “Friendship and Mobility” paper (https://snap.stanford.edu/data/).
“Global-scale Check-in Dataset with User Social Networks” from two research projects at this address (project 5 by Dingqi Yang): ( https://sites.google.com/site/yangdingqi/home/foursquare-dataset#h.p_7rmPjnwFGIx9). The dataset is coming from Foursquare and it contains the information of 22,809,624 checkins by 114,324 users, 607,333 friendship links and 3,820,891 POIs. It is approximately twice as big as the other dataset. It also contains a set of worldwide check-ins with country flags taken over about two years and two snapshots of the corresponding user social network before (in Mar. 2012) and after (in May 2014) the check-in data collection period. In order to use it, we will need to merge and compare these different databases to transform it to obtain a dataset that looks like the one of Gowalla and Brightkite.

---
### Structure of this notebook

To be defined

---
### Preliminary thoughts

The data wrangling part would be a hard part, since our datasets are very large : we need to be smart in order not to crash our kernel.

### Step 1)
We want to assign a nationality to each user. In order to do so, we decide that a user has a set nationality if it checked in more than 5 times in a given country. In case there are two or more countries where our user checked in a lot of times, we take the country with the most check-ins as nationality.

**Question 1)** Are there countries with no inhabitants because nobody checked-in 5 times ? If yes, we exclude these countries for the moment and will look at them later when we focus on holiday destinations. <br>
**Question 2)** Are there users that have checked-in less than 5 times in every country and that therefore have no nationality according to our criterion ? <br>
**Question 3)** Assign a nationality to each user !

In [41]:
import pandas as pd
from zipfile import ZipFile
import datetime

### Question 1)

In [42]:
# We start by extracting our data from our zip file.
zip_file = ZipFile('dataset_WWW2019.zip')
my_df = pd.read_csv(zip_file.open('dataset_WWW2019/raw_POIs.txt'), header = None, sep = "\t", error_bad_lines=False, names = ["zone_id","Lat","lon","building","country"])
my_df

,zone_id,Lat,lon,building,country
0,3fd66200f964a52000e61ee3,40.729209,-73.998753,Post Office,US
1,3fd66200f964a52000e71ee3,40.733596,-74.003139,Jazz Club,US
2,3fd66200f964a52000e81ee3,40.758102,-73.975734,Gym,US
3,3fd66200f964a52000ea1ee3,40.732456,-74.003755,Indian Restaurant,US
4,3fd66200f964a52000ec1ee3,42.345907,-71.087001,Indian Restaurant,US
...,...,...,...,...,...
11180155,52e92ec0498e1a09484475c9,55.852631,37.447034,Residential Building (Apartment / Condo),RU
11180156,52e92f11498e8e25136e9fe2,-4.329570,-40.711533,Dentist's Office,BR
11180157,52e92f4511d2a77832a49eec,52.505471,13.368507,Financial or Legal Service,DE
11180158,52e92fa5498ef926067570ff,41.025367,28.554854,City,TR


In [43]:
# Since we want to study our data by country, we get the list of all different countries we can study.
list_country = list(my_df.country.unique()) # list of all countries where people have checked in.

# We also create a dataframe where we order each country in accordance to the number of check-ins it holds.
my_countries = my_df.groupby(by = "country").agg("count").sort_values(by = "building", ascending = False).drop(columns = ["zone_id", "Lat", "lon"])
my_countries[my_countries["building"] >=200000]

# We notice that a lot of countries have a few check-ins. We don't know if it's because somebody from another contry
# checked-in once while on holiday or if we have users that check-in occasionely.
# However, since at this stage we just want to get nationality, we surpress countries with too few checkins.

,building
country,
US,1990327
ID,1198611
BR,1159258
TR,1098373
RU,546532
JP,519409
MY,493453
MX,408434
TH,353444


In [44]:
info = my_countries.reset_index()
print(info[info["country"] == "FR"])
print(info[info["country"] == "CH"])

   country  building
18      FR    108269
   country  building
46      CH     19597


In [45]:
countries_with_five_checkins = my_countries[my_countries["building"] >=5].index.tolist()
countries_to_exclude = [element for element in my_countries.index if element not in countries_with_five_checkins]
countries_to_exclude 

['SH',
 'KM',
 'TV',
 'TF',
 'ST',
 'UM',
 'CS',
 'CF',
 'NR',
 'ER',
 'CX',
 'PM',
 'TA',
 'GS',
 'BQ',
 'WF',
 'JT',
 'NU',
 'FX',
 'DD',
 'CP']

In [46]:
my_df = my_df[my_df["country"].isin(countries_with_five_checkins)]
my_df
# We notice that removing our countries with a low number of check-ins doesn't change our dataset much.  

,zone_id,Lat,lon,building,country
0,3fd66200f964a52000e61ee3,40.729209,-73.998753,Post Office,US
1,3fd66200f964a52000e71ee3,40.733596,-74.003139,Jazz Club,US
2,3fd66200f964a52000e81ee3,40.758102,-73.975734,Gym,US
3,3fd66200f964a52000ea1ee3,40.732456,-74.003755,Indian Restaurant,US
4,3fd66200f964a52000ec1ee3,42.345907,-71.087001,Indian Restaurant,US
...,...,...,...,...,...
11180155,52e92ec0498e1a09484475c9,55.852631,37.447034,Residential Building (Apartment / Condo),RU
11180156,52e92f11498e8e25136e9fe2,-4.329570,-40.711533,Dentist's Office,BR
11180157,52e92f4511d2a77832a49eec,52.505471,13.368507,Financial or Legal Service,DE
11180158,52e92fa5498ef926067570ff,41.025367,28.554854,City,TR


### Question 2)

In [47]:
# We now get our dataset of checkins. This dataset does not contain the name of the country. It however contains
# Indirect information. 
my_df_checkins = pd.read_csv(zip_file.open('dataset_WWW2019/dataset_WWW_Checkins_anonymized.txt'), header = None, sep = "\t", error_bad_lines=False, names = ["person_id","zone_id","time_checkin","timezone"])
my_df_checkins

,person_id,zone_id,time_checkin,timezone
0,822121,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180
1,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300
2,113817,4b4bade2f964a520cfa326e3,Tue Apr 03 18:00:09 +0000 2012,-240
3,14732,4c143cada5eb76b0dc7dc1b7,Tue Apr 03 18:00:09 +0000 2012,-240
4,1397630,4e88cf4ed22d53877981fdab,Tue Apr 03 18:00:09 +0000 2012,-300
...,...,...,...,...
22809619,1568872,502bb1cee4b0190fdc74767b,Wed Jan 29 16:43:46 +0000 2014,420
22809620,689856,4d701b20b73bb1f70867b472,Wed Jan 29 16:43:56 +0000 2014,-180
22809621,567861,4b51b5ecf964a5203e5327e3,Wed Jan 29 16:44:04 +0000 2014,-120
22809622,44698,504a21c3e4b0568d3c5e6794,Wed Jan 29 16:44:16 +0000 2014,-360


In [48]:
# Our participants checked-in at least 73 times in foursquare. As a result, we don't have to exclude any user from
# our dataset.
my_participants = my_df_checkins.groupby(by = "person_id").agg("count")
my_participants.min()

zone_id         73
time_checkin    73
timezone        73
dtype: int64

### Question 3)

In [49]:
def extract_year(country_code, my_year, Checkin_dataframe, POI_dataframe, timezone_low, timezone_high = None):
    ### Function to extract a dataframe for a given country in a given year. The returned dataframe is of the form : 
    ###  index = zone_id	person_id 	time_checkin 	year 	Lat 	lon 	building 	country
    ### time_checkin and year are of type pandas._libs.tslibs.timestamps.Timestamp
    ### In order to extract one country, you have to give "country_code" (ex : "US"), "my_year" (a str ex : "2013")
    ### The dataframe of all checkins, the dataframe of all POI and the timezone of the country in minutes
    ### (ex : for the US, the timezone is timezone_low = -480, timezone_high = -300. If you have only one timezone
    ### like GB who only has timezone_low = 0, don't give an argument for timezone_high).
    ### We have to look on the internet to find the timezone.
    
    ### IMPORTANT : the time complexity of this function is very much linked to the number of timezones in a country. 
    
    if timezone_high is None:
        timezone_high = timezone_low + 1
        
    # We focus only on one country, we extract all the POI of this country.
    country_POI = POI_dataframe[POI_dataframe["country"] == country_code]
    
    # We think it could be a bit long to merge our two datasets on 22 million entries. As a result, we do a first
    # verification : a user in a given country if he doesn't match the timezone of the country.
    Checkins_in_timezone = Checkin_dataframe[Checkin_dataframe.timezone.between(timezone_low, timezone_high)].drop("timezone", axis = 1)
    
    # We extract the year we are interested in.
    Checkins_in_timezone["time_checkin"] = pd.to_datetime(Checkins_in_timezone["time_checkin"], format = "%a %b %d %X %z %Y")
    Checkins_in_timezone["year"] = pd.DatetimeIndex(Checkins_in_timezone["time_checkin"]).year
    Checkins_in_year = Checkins_in_timezone[Checkins_in_timezone["year"] == pd.to_datetime(my_year).year]
    
    # Finally, we merge our datasets.
    merge_checkins = pd.merge(Checkins_in_year.set_index("zone_id"), country_POI.set_index("zone_id"), left_index=True, right_index=True)
    return(merge_checkins)

In [50]:
GB_db = extract_year("GB", "2013", my_df_checkins, my_df, 0)

In [51]:
GB_db

,person_id,time_checkin,year,Lat,lon,building,country
zone_id,,,,,,,
4aa37fb3f964a520f64320e3,576823,2013-01-01 14:14:12+00:00,2013,55.948431,-3.199811,Historic Site,GB
4aa37fb3f964a520f64320e3,283600,2013-01-02 16:18:11+00:00,2013,55.948431,-3.199811,Historic Site,GB
4aa37fb3f964a520f64320e3,2017013,2013-01-27 09:58:02+00:00,2013,55.948431,-3.199811,Historic Site,GB
4aa37fb3f964a520f64320e3,18802,2013-03-02 13:38:34+00:00,2013,55.948431,-3.199811,Historic Site,GB
4aa37fb3f964a520f64320e3,1935653,2013-03-02 14:13:54+00:00,2013,55.948431,-3.199811,Historic Site,GB
...,...,...,...,...,...,...,...
52b30990498ed3abb6377240,169786,2013-12-19 14:58:44+00:00,2013,52.874711,-1.544809,Assisted Living,GB
52b31411498e2a7e3c07161a,376722,2013-12-19 15:45:31+00:00,2013,51.538240,0.051458,Furniture / Home Store,GB
52b41cba498e07336dc4b3ac,211455,2013-12-20 10:32:57+00:00,2013,51.781743,-0.188854,Hospital,GB


A first interesting fact is that we have more data in 2012 than any other year. This is surprising since we don't even have the data for a full year !<br>
We have less data in 2013 though we have the full year data. This can be explained by the fact that foursquare's popularity was starting to waver and would only grow again towards 2014 with the release of "swarm", a companion app to foursquare. <br>
In order to have a coherent dataset, we will start by working on data from 2013 (where we have the full year) and check our results by using data from 2012 and 2014.

In [52]:
GB_db.to_csv("GB_2013_merge_data.csv", index = True)

In [53]:
test = pd.read_csv("GB_2013_merge_data.csv")

In [54]:
test

,zone_id,person_id,time_checkin,year,Lat,lon,building,country
0,4aa37fb3f964a520f64320e3,576823,2013-01-01 14:14:12+00:00,2013,55.948431,-3.199811,Historic Site,GB
1,4aa37fb3f964a520f64320e3,283600,2013-01-02 16:18:11+00:00,2013,55.948431,-3.199811,Historic Site,GB
2,4aa37fb3f964a520f64320e3,2017013,2013-01-27 09:58:02+00:00,2013,55.948431,-3.199811,Historic Site,GB
3,4aa37fb3f964a520f64320e3,18802,2013-03-02 13:38:34+00:00,2013,55.948431,-3.199811,Historic Site,GB
4,4aa37fb3f964a520f64320e3,1935653,2013-03-02 14:13:54+00:00,2013,55.948431,-3.199811,Historic Site,GB
...,...,...,...,...,...,...,...,...
60686,52b30990498ed3abb6377240,169786,2013-12-19 14:58:44+00:00,2013,52.874711,-1.544809,Assisted Living,GB
60687,52b31411498e2a7e3c07161a,376722,2013-12-19 15:45:31+00:00,2013,51.538240,0.051458,Furniture / Home Store,GB
60688,52b41cba498e07336dc4b3ac,211455,2013-12-20 10:32:57+00:00,2013,51.781743,-0.188854,Hospital,GB
60689,52b423e4498e87320c362e9a,1924356,2013-12-20 11:03:17+00:00,2013,54.816442,-1.627189,Nursery School,GB


### Other steps :
Vacation destinations ? Favourite place to go per country ? Number of friends per country (Put in relationship with total number of Foursquare user) ? Number of checkins per user/per country (Use this to compute uncertainty for each statistics we may find) ? Seasonality of check-ins ? When do people do on holidays ? (Which seasons)...

In [55]:
ID_df = pd.read_csv("ID_2013_merge_data.csv")

In [56]:
ID_df

,zone_id,person_id,time_checkin,year,Lat,lon,building,country
0,4b05880ff964a5201caf22e3,1170327,2013-01-02 06:04:37+00:00,2013,1.145077,104.010916,Other Great Outdoors,ID
1,4b05880ff964a5201caf22e3,1170327,2013-01-11 11:31:56+00:00,2013,1.145077,104.010916,Other Great Outdoors,ID
2,4b05880ff964a5201caf22e3,1170327,2013-01-15 10:18:07+00:00,2013,1.145077,104.010916,Other Great Outdoors,ID
3,4b05880ff964a5201caf22e3,408433,2013-01-30 11:12:24+00:00,2013,1.145077,104.010916,Other Great Outdoors,ID
4,4b05880ff964a5201caf22e3,408433,2013-01-30 11:25:09+00:00,2013,1.145077,104.010916,Other Great Outdoors,ID
...,...,...,...,...,...,...,...,...
771866,52b6c7f611d255fa1a7392a3,707704,2013-12-22 11:08:20+00:00,2013,0.475714,121.241539,Rest Area,ID
771867,52b6ca4e498e15b460e38ab2,683822,2013-12-22 11:17:49+00:00,2013,-6.227489,106.825379,Australian Restaurant,ID
771868,52b6cada11d248b7b56d2374,1264982,2013-12-22 11:20:12+00:00,2013,-8.569188,114.090425,Home (private),ID
771869,52b6cd3e498ea6b8c72aa057,1499509,2013-12-22 11:46:16+00:00,2013,1.459921,124.823780,Home (private),ID


### Preprocessing


In [57]:
#Helpers
def dropLocationId(df):
    return df.drop(['zone_id'],axis=1)

def dropCheckinTime(df):
    return df.drop(['time_checkin'],axis=1)

def dropMissingLatOrLng(df):
    return df[~((df['Lat'] == 0) | (df['lon'] == 0))]

def dropInvalidLat(df):
    return df[(df['Lat'] < 90) & (df['Lat'] > -90)]

def dropInvalidLng(df):
    return df[(df['lon'] < 180) & (df['lon'] > -180)]

def preprocess(df):
    df_ = df.copy()
    #drop Rows having NaN/Null/NaT Values 
    df_ = df_.dropna(how="any")

    #drop Rows having invalid Values 
    df_ = dropMissingLatOrLng(df_)

    df_ = dropInvalidLat(df_)
    df_ = dropInvalidLng(df_)
    return df_
    


In [58]:
totalChekinsBefore = ID_df.shape[0]

In [59]:
clean_df = preprocess(ID_df)

In [60]:

#Total number of check-ins 
print('Before the preprocessing, the dataset had : '+ str(totalChekinsBefore) + ' checkins')
print('After the preprocessing, the dataset had : '+ str(clean_df.shape[0]) + ' checkins')


Before the preprocessing, the dataset had : 771871 checkins
After the preprocessing, the dataset had : 771871 checkins


In [61]:
#Splitting per zone

SyntaxError: invalid syntax (<ipython-input-61-5a3754c44f04>, line 1)